**In this project, I will conduct an `Explanatory analysis(EDA)` on the dataset <a href="https://www.kaggle.com/search?q=ultra+marathon+in%3Adatasets"> The big dataset of ultra-marathon running</a>,  which is available on Kaggle. Although my dataset is not exceptionally large (around 1 gigabyte in CSV format), it is still significant. My aim is to perform data manipulation and EDA on this large dataset using the `Pandas` library.**

In [1]:
import pandas as pd
import seaborn as sns

In [3]:
data = pd.read_csv("clean_dataset.csv")
data.head(2)

C:\Users\saeed\AppData\Local\Temp\ipykernel_19284\4232497205.py:1: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("clean_dataset.csv")


,year_of_event,event_dates,event_name,event_distance/length(km),event_number_of_finishers,athlete_performance,athlete_club,athlete_country,athlete_year_of_birth,athlete_gender,athlete_age,athlete_average_speed,athlete_id
0,2018,06.01.2018,6 Stunden Self-Transcendence Sri Chinmoy Smole...,60.375,9,6.0,Мкшч,RUS,1966.0,M,50,10.063,22
1,2018,06.01.2018,6 Stunden Self-Transcendence Sri Chinmoy Smole...,59.750,9,6.0,Лично,RUS,1990.0,M,23,9.958,23


---